# CNN

## Loading the data

We loaded the data directly from Google Drive so we have to import drive from google.colab

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tensorflow as tf
import numpy as np

# Set the seed for random operations. 
# This let our experiments to be reproducible. 
SEED = 1234
tf.random.set_seed(SEED)  

In [ ]:
# ImageDataGenerator
# ------------------

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_data_gen = ImageDataGenerator(rescale=1./255)
valid_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Create generators to read images from dataset directory
# -------------------------------------------------------

# img

# Batch size
bs = 64

# img shape
img_h = 32
img_w = 32

num_classes=5

# Training
training_dir = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/datasets/CIFAR10_dataset/train_40k_10k/train_4k'
train_gen = train_data_gen.flow_from_directory(training_dir,
                                               target_size = (32, 32),
                                               color_mode='rgb',
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)  # targets are directly converted into one-hot vectors

# Validation
validation_dir = '/content/drive/MyDrive/CIFAR10_CNN_validations/Val_4k_1k'
valid_gen = valid_data_gen.flow_from_directory(validation_dir,
                                               target_size = (32, 32),
                                               color_mode='rgb',
                                               batch_size=bs, 
                                               class_mode='categorical',
                                               shuffle=False,
                                               seed=SEED)


# Test
test_dir = '/content/drive/MyDrive/colab-sg2-ada/stylegan2-ada/datasets/CIFAR10_dataset/test/test_5k'
test_gen = test_data_gen.flow_from_directory(test_dir,
                                             target_size = (32, 32),
                                             color_mode='rgb',
                                             batch_size=bs, 
                                             class_mode='categorical',
                                             shuffle=False,
                                             seed=SEED)


Found 20096 images belonging to 5 classes.
Found 5040 images belonging to 5 classes.
Found 5010 images belonging to 5 classes.


In [ ]:
# Create Dataset objects
# ----------------------

# Training
train_dataset = tf.data.Dataset.from_generator(lambda: train_gen,
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, 32, 32, 3], [None, 5]))

# Repeat
train_dataset = train_dataset.repeat()

# Validation
# ----------
valid_dataset = tf.data.Dataset.from_generator(lambda: valid_gen, 
                                               output_types=(tf.float32, tf.float32),
                                               output_shapes=([None, 32, 32, 3], [None, 5]))

# Repeat
valid_dataset = valid_dataset.repeat()


# Test
# ----
test_dataset = tf.data.Dataset.from_generator(lambda: test_gen,
                                              output_types=(tf.float32, tf.float32),
                                              output_shapes=([None, 32, 32, 3], [None, 5]))

## CNN Model

In [ ]:
model_cnn = tf.keras.Sequential()

model_cnn.add(tf.keras.layers.Conv2D(input_shape=(32, 32, 3),
                                 filters=32,
                                 kernel_size=(3, 3),        
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.Conv2D(input_shape=(32, 32, 3),
                                 filters=32,
                                 kernel_size=(3, 3),        
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Conv2D(filters=64,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.Conv2D(filters=64,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Conv2D(filters=128,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.Conv2D(filters=128,
                                 kernel_size=(3, 3),
                                 strides=(1, 1), 
                                 padding='same'))
model_cnn.add(tf.keras.layers.ReLU())
model_cnn.add(tf.keras.layers.BatchNormalization())
model_cnn.add(tf.keras.layers.MaxPool2D(pool_size=(2, 2)))

model_cnn.add(tf.keras.layers.Flatten())
model_cnn.add(tf.keras.layers.Dropout(0.2))

model_cnn.add(tf.keras.layers.Dense(units=1024, activation=tf.keras.activations.sigmoid))
model_cnn.add(tf.keras.layers.Dropout(0.2)) 

model_cnn.add(tf.keras.layers.Dense(units=5, activation=tf.keras.activations.softmax))

model_cnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 32, 32, 32)        896       
                                                                 
 re_lu (ReLU)                (None, 32, 32, 32)        0         
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 32)        9248      
                                                                 
 re_lu_1 (ReLU)              (None, 32, 32, 32)        0         
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 32)       128       
 hNormalization)                                        

In [ ]:
model_cnn.compile(optimizer='adam',loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics = ['accuracy']) 

In [ ]:
model_cnn.fit(
    x = train_dataset,
    epochs=150,
    steps_per_epoch=int(np.ceil(train_gen.samples / bs)),
    validation_data= valid_dataset,
    validation_steps=int(np.ceil(valid_gen.samples / bs))
    )

In [ ]:
# Save the entire model as a SavedModel.
model_cnn.save_weights('/content/drive/MyDrive/CNN_Models_results/CNN.h5')